# Simulation Truth

This notebook will introduce you to the concept of simulation truth in fuse.



## Imports and Simulation Context

Similar to the previous notebooks, we will start by importing the necessary modules and creating a simulation context. Additional we register two new plugins called `PeakTruth` and `SurvivingClusters`.

In [1]:
import fuse
import numpy as np

/home/hschulze/.local/lib/python3.9/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/home/hschulze/.local/lib/python3.9/site-packages/straxen/plugins/led_cal/led_calibration.py:232: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def calc_delta_time(ext_timings_nv_delta_time, pulses, hitlets_nv, nv_pmt_start, nv_pmt_stop):


In [2]:
st = fuse.context.full_chain_context(output_folder = "./fuse_data")

st.register(fuse.plugins.truth_information.PeakTruth)
st.register(fuse.plugins.truth_information.SurvivingClusters)

st.set_config({"path": "/project2/lgrandi/xenonnt/simulations/testing",
               "file_name": "pmt_neutrons_100.root",
               "entry_stop": 10,
               })

run_number = "00000"

Warning! elife not in context config, skipping...
Warning! electron_drift_velocity not in context config, skipping...
Warning! electron_drift_time_gate not in context config, skipping...


## Raw_Records and Contributing_Clusters

First we will run the simulation up to `raw_records`. The `PMTResponseAndDAQ` plugin now has two outputs, `raw_records` and `contributing_channels`, both are saved to disk when we request fuse to produce `raw_records`.

In [3]:
st.make(run_number, "microphysics_summary")
st.make(run_number, "raw_records")

/home/hschulze/.local/lib/python3.9/site-packages/straxen/url_config.py:743: UserWarning: From straxen version 2.1.0 onward, URLConfig parameterswill be sorted alphabetically before being passed to the plugins, this will change the lineage hash for non-sorted URLs. To load data processed with non-sorted URLs, you will need to use an older version.
  warnings.warn(


Now that the data is produced, lets load it. Both are of the same `data_kind` so we can load them together.

In [4]:
raw_records = st.get_array(run_number, ["raw_records", "contributing_clusters"])

Loading plugins: |          | 0.00 % [00:00<?]

`contributing_clusters` gives you five additional columns. These are:
- `contributing_clusters` - A list of the clusters that contributed to the `raw_record`
- `s1_photons_per_cluster` - The number of S1 photons that of the corresponding cluster in the `raw_record`
- `s2_photons_per_cluster` - The number of S2 photons that of the corresponding cluster in the `raw_record`
- `ap_photons_per_cluster` - The number of (virtual) PMT afterpulse 'photons'
- `raw_area` - The sum of the contributing photon gains divided by the gain of the PMT

Lets have a look what clusters contributed to the first record:

In [5]:
print(raw_records[0]["contributing_clusters"])

[1 0 0 0 0]


You can see that we get a list of length 5. This is a compromise we need to make as we can't store a list of variable length in a strax. In this case we only store the information of the 5 first clusters that contributed to the record. If there are more than 5 clusters for one record, this information is lost. For simulations with a lot of clusters per event, it makes sense to increase the number of clusters that are stored per record. This can be done by changing the config option `max_contributing_channels_in_truth` of the `PMTResponseAndDAQ` plugin. Lets try this out:

In [6]:
st.set_config({"max_contributing_channels_in_truth": 35,})
st.make(run_number, "raw_records")

In [7]:
raw_records = st.get_array(run_number, ["raw_records", "contributing_clusters"])

Loading plugins: |          | 0.00 % [00:00<?]

In [8]:
index = np.argmax(np.sum(raw_records["contributing_clusters"]>0, axis = 1))

print(raw_records[index]["contributing_clusters"])

[  3   4   7   8  16  17  18  19  21  30  31  32  38  45  47  51  53  56
  58  61  62  63  64  69  74  79  80  86  87  95  97  98 104 106 112]


Now the list is 35 elements long, just as we requested. Depending on the source this might still be not enough. We can now have a look with how many photons each of these clusters contributed to the record:

In [9]:
print("S1 photons:", raw_records[index]["s1_photons_per_cluster"])
print("S2 photons:", raw_records[index]["s2_photons_per_cluster"])
print("AP photons:", raw_records[index]["ap_photons_per_cluster"])

S1 photons: [1 1 1 1 1 1 3 4 3 1 1 1 1 1 2 2 4 1 1 1 1 1 1 1 1 1 3 1 1 2 1 1 2 1 1]
S2 photons: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
AP photons: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


We can have a look at the clusters that contributed to the record. `microphysics_summary` contains the column `cluster_id`. This is the same number as stored in `contributing_clusters`. Please note that `cluster_id` is only unique per chunk of data.

In [10]:
microphysics_summary = st.get_df(run_number, "microphysics_summary")

Loading microphysics_summary: |          | 0.00 % [00:00<?]

In [11]:
microphysics_summary[np.isin(microphysics_summary.cluster_id.values, raw_records[index]["contributing_clusters"])].head()

,e_field,time,endtime,x,y,z,ed,nestid,A,Z,...,x_pri,y_pri,z_pri,cluster_id,xe_density,vol_id,create_S2,photons,electrons,excitons
6,28,2827386343,2827386343,-38.823586,-18.264086,-6.666750,35.142311,8,0,0,...,-44.154747,2.803934,7.933488,3,2.862,1,True,2275,385,405
7,28,2827386343,2827386343,-38.822685,-18.261517,-6.661916,73.418549,8,0,0,...,-44.154747,2.803934,7.933488,4,2.862,1,True,4412,992,813
10,28,2827386343,2827386343,-38.826496,-18.252813,-6.656171,16.106411,8,0,0,...,-44.154747,2.803934,7.933488,7,2.862,1,True,837,328,138
11,28,2827386343,2827386343,-38.827171,-18.249474,-6.656644,8.190919,8,0,0,...,-44.154747,2.803934,7.933488,8,2.862,1,True,415,181,48
18,24,2827386343,2827386343,-12.585729,17.032082,-36.443317,257.285248,8,0,0,...,-44.154747,2.803934,7.933488,16,2.862,1,True,12707,6230,2826


## Peaks and peak_truth

Next we can process the simulation result to `peak_basics`. Strax(en) will merge multiple records into a peak. The PeakTruth plugin will evaluate which `raw_records` contribute to a peak and calculate a truth output for each peak. The provided columns for each peak are:
- `s1_photon_number_truth` - The number of S1 photons that contributed to the peak
- `s2_photon_number_truth` - The number of S2 photons that contributed to the peak
- `ap_photon_number_truth` - The number of (virtual) PMT afterpulse 'photons' that contributed to the peak
- `raw_area_truth` - The sum of all contributing photon gains divided by the gains of the PMTs
- `observable_energy_truth` - Estimate of the energy that is associated with the peak
- `number_of_contributing_clusters` - Number of clusters that contributed to the peak
- `average_x_of_contributing_clusters` - Weighted average of the x position of the clusters that contributed to the peak
- `average_y_of_contributing_clusters` - Weighted average of the y position of the clusters that contributed to the peak
- `average_z_of_contributing_clusters` - Weighted average of the z position of the clusters that contributed to the peak


In [12]:
st.make(run_number, "peak_truth")
st.make(run_number, "peak_positions")

As strax(en) will take care of the matching of our truth information to the individual peaks, we can simply load the `peak_basics` and `peak_truth` data together.

In [13]:
peak_basics = st.get_df(run_number, ["peak_basics", "peak_truth", "peak_positions"])

Loading plugins: |          | 0.00 % [00:00<?]

For a peak area bias study we could now compare the raw_area to the peak area:

In [14]:
peak_basics[["area", "raw_area_truth"]].head()

,area,raw_area_truth
0,3.653952,3.689999
1,959.812744,964.619812
2,61777.117188,61782.406250
3,855.671936,860.039856
4,306587.468750,294382.250000


We might also be interested in the peak classification: 

In [15]:
peak_basics[["type", "s1_photon_number_truth", "s2_photon_number_truth", "ap_photon_number_truth"]].head()

,type,s1_photon_number_truth,s2_photon_number_truth,ap_photon_number_truth
0,1,3,0,0
1,2,0,785,0
2,1,29885,0,55
3,2,0,678,1
4,2,0,232998,247


Or you might want to check how our position reconstruction is doing: 

In [16]:
peak_basics[["type","x","y", "average_x_of_contributing_clusters", "average_y_of_contributing_clusters", "average_z_of_contributing_clusters"]].head()

,type,x,y,average_x_of_contributing_clusters,average_y_of_contributing_clusters,average_z_of_contributing_clusters
0,1,NaN,NaN,-20.890713,-51.367107,-1.464800
1,2,-20.734354,-51.598339,-20.890713,-51.367107,-1.464800
2,1,-59.119907,5.821635,-19.649834,6.006746,-28.128731
3,2,-37.885193,-18.474636,-38.867641,-18.246555,-6.679745
4,2,-37.515583,-19.312231,-35.237564,-15.965981,-10.668817


## Surviving Clusters
Finally we can evaluate if an energy deposit makes it into a record or a peak. This is done by the `SurvivingClusters` plugin. It will provide the following columns:
- `in_a_record` - Boolean if the cluster is in a record
- `in_a_peak` - Boolean if the cluster is in a peak

In [21]:
st.make(run_number, "surviving_clusters")
microphysics_summary = st.get_df(run_number, ["microphysics_summary", "surviving_clusters"])

Loading plugins: |          | 0.00 % [00:00<?]

Now that we have the data loaded we could have a look at clusters that did not make it into a peak: 

In [23]:
microphysics_summary.query("in_a_peak == False").head()

,e_field,time,endtime,x,y,z,ed,nestid,A,Z,...,z_pri,cluster_id,xe_density,vol_id,create_S2,photons,electrons,excitons,in_a_record,in_a_peak
170,27,2827386497,2827386497,32.959923,14.185023,-9.616013,0.096808,0,0,0,...,7.933488,171,2.862,1,True,0,0,0,False,False
203,26,2827386535,2827386535,27.667393,10.273334,-12.147082,0.044494,0,0,0,...,7.933488,204,2.862,1,True,0,0,0,False,False
205,25,2827386710,2827386710,9.488111,6.251637,-11.284761,0.141653,0,0,0,...,7.933488,206,2.862,1,True,0,0,0,False,False
208,24,2827386851,2827386851,16.457802,8.368632,-30.002600,0.085379,0,0,0,...,7.933488,209,2.862,1,True,0,0,0,False,False
209,24,2827386893,2827386893,16.093048,14.685450,-34.862846,0.057414,0,0,0,...,7.933488,210,2.862,1,True,0,0,0,False,False
